In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
#New Syntax
from selenium.webdriver.chrome.service import Service as Service
service = Service(executable_path ='chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [3]:
try:
    button = driver.find_element_by_xpath('//*[@id="tarteaucitronPersonalize2"]')
    button.click()
except:
    pass

In [4]:
from dags.functions.scraping_superimmo_functions import get_details

In [14]:
driver.get('https://www.superimmo.com/achat/aquitaine/gironde?option%5B%5D=old_build')
max_pages = int(driver.find_element(By.XPATH, '//*[@id="pjax-container"]/div[2]/nav/ul/li[10]/a').text)

df = pd.DataFrame()

ind = 0

for page in tqdm(range(10)):

    if page == 1:
        page = 0
    pages_url = f'https://www.superimmo.com/achat/aquitaine/gironde/p/{page}?option%5B%5D=old_build'
    driver.get(pages_url)
    
    for i in tqdm(range(1, 16), leave = False):
        
        driver.get(pages_url)
        url = driver.find_element(By.XPATH, f'//*[@id="pjax-container"]/section/article[{i}]/section/div[5]/p/a').get_attribute("href")
        price, surface, nb_pieces, nb_chambres, location, prix_m2, date_publication, tag_1, tag_2, tag_3, description, tags, images_url = get_details(url)

        new_row = pd.DataFrame({'url': url, 'price': price, 'surface': surface, 'nb_pieces': nb_pieces,
                                         'nb_chambres': nb_chambres, 'lieu': location, 'prix_m2': prix_m2,
                                         'date_publication': date_publication, 'tag_1': tag_1, 'tag_2': tag_2,
                                         'tag_3': tag_3, 'description': description, 
                                         'tags': [tags], 'images_url': [images_url]})

        df = pd.concat([df, new_row], ignore_index = True)

df.head()

  0%|          | 0/10 [00:18<?, ?it/s]


KeyboardInterrupt: 

'93 075'

In [5]:
# Get URL of annonces
def get_individual_url(pages_url):
    driver.get(pages_url)
    
#    for i in range(1, 16):
    for i in range(1, 2):
        url = driver.find_element(By.XPATH, f'//*[@id="pjax-container"]/section/article[{i}]/section/div[5]/p/a').get_attribute("href")
        print(url)

In [11]:
driver.get("https://www.superimmo.com/annonces/achat-appartement-merignac-33700-xw9fd9")

prix_m2 = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/div[1]/ul/li[1]').text
prix_m2 = re.findall(r'(\d* *\d+) €\/m²', prix_m2)[0]
prix_m2 = int(prix_m2.replace(" ", ""))

print(prix_m2)

IndexError: list index out of range

In [14]:
prix_m2 = "Exclu"
if re.search(r'(\d* *\d+) €\/m²', prix_m2):
    print("ok")
else:
    print("ko")

ko


In [76]:
driver.get("https://www.superimmo.com/annonces/achat-appartement-paris-2eme-75002-xw8evz")
prop = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/div[1]/div[2]/div/h1/div/div[4]').text

print(prop)

if re.search(r"([\wéè]+)\s*\((\d+)\)", prop):
    print(prop)

Paris 2ème (75002)
Paris 2ème (75002)


In [68]:
def find_property(x_path, surface, nb_pieces, nb_chambres, location):

    property = driver.find_element(By.XPATH, x_path).text

    if re.search(r"m²", property):
        surface = re.findall(r'(\d+)* m²', property)[0]
        surface = int(surface.replace(" ", ""))
    elif re.search(r"pièce", property):
        nb_pieces = re.findall(r'(\d+)', property)[0]
        nb_pieces = int(nb_pieces.replace(" ", ""))
    elif re.search(r"chambre", property):
        nb_chambres = re.findall(r'(\d+)', property)[0]
        nb_chambres = int(nb_chambres.replace(" ", ""))
    elif re.search(r"([\wéè]+)\s*\((\d+)\)", property):
        location = property

    return surface, nb_pieces, nb_chambres, location

In [77]:
def get_details(url, driver):
    driver.get(url)
    
    price, surface, nb_pieces, nb_chambres, location, prix_m2, date_publication = '', '', '', '', '', '', ''
    tag_1, tag_2, tag_3, description = '', '', '', ''
    images_url = [None] * 10
    tags = [None] * 20

    # Recherche du prix de vente
    try:
        price = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[2]/section[1]/table/tbody/tr/td[1]').text
        price = re.findall(r'Prix de vente : (\d* *\d+ \d+) €', price)[0]
        price = int(price.replace(" ", ""))
    except:
        pass

    # Etude de chaque bloc en-dessous de l'image principale de l'annonce
    for xpath in ['/html/body/main/div[2]/div[1]/div[2]/div[2]/div/h1/div/div[1]',
                  '/html/body/main/div[2]/div[1]/div[1]/div[2]/div/h1/div/div[1]',
                  '/html/body/main/div[2]/div[1]/div[2]/div[2]/div/h1/div/div[2]',
                  '/html/body/main/div[2]/div[1]/div[1]/div[2]/div/h1/div/div[2]',
                  '/html/body/main/div[2]/div[1]/div[2]/div[2]/div/h1/div/div[3]',
                  '/html/body/main/div[2]/div[1]/div[1]/div[2]/div/h1/div/div[3]',
                  '/html/body/main/div[2]/div[1]/div[2]/div[2]/div/h1/div/div[4]',
                  '/html/body/main/div[2]/div[1]/div[1]/div[2]/div/h1/div/div[4]',
                  '/html/body/main/div[2]/div[1]/div[2]/div[2]/div/h1/div/div[5]',
                  '/html/body/main/div[2]/div[1]/div[1]/div[2]/div/h1/div/div[5]',
                ]:  
        try:
            surface, nb_pieces, nb_chambres, location = find_property(xpath, surface, nb_pieces, nb_chambres, location)        
        except:
            pass
        
    # Recherche des autres propriétés
    try:
        prix_m2 = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/div[1]/ul/li[1]').text
        prix_m2 = re.findall(r'(\d* *\d+) €\/m²', prix_m2)[0]
        prix_m2 = int(prix_m2.replace(" ", ""))
    except:
        pass

    try:
        date_publication = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[2]/div[1]').text
        date_publication = re.findall(r'Publiée le (\d{2}/\d{2}/\d{4})', date_publication)
    except:
        pass

    try:
        tag_1 = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/div[1]/ul/li[2]').text
        tag_2 = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/div[1]/ul/li[3]').text
    except:
        pass

    try:
        tag_3 = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[2]/div[3]/p').text
    except:
        pass


    for i in range(1, 11):
        for j in range(1, 3):
            try:
                tag = driver.find_element(By.XPATH, f'/html/body/main/div[2]/div[2]/section[2]/table/tbody/tr[{i}]/td[{j}]').text
                tags[(i-1) * 2 + j - 1] = tag
            except:
                pass

    try:
        description = driver.find_element(By.XPATH, '/html/body/main/div[2]/div[2]/p').text
    except:
        pass
    
    try:
        nb_images = int(driver.find_element(By.XPATH, '/html/body/main/div[2]/div[1]/div[1]/div[1]/div[1]/span').text)
        for img in range(1, nb_images + 1):
            url = driver.find_element(By.XPATH, f'/html/body/main/div[2]/div[1]/div[1]/div[1]/div[2]/div[{img}]/span/img').get_attribute("src")
            images_url[img - 1] = url
    except:
        pass

    print(
        "Price:", price, "\n",  
        "Surface:", surface, "\n", 
        "Nb pieces:", nb_pieces, "\n", 
        "Nb chambres:", nb_chambres, "\n", 
        "Location:", location, "\n", 
        "Prix m2:", prix_m2, "\n", 
        "Date Publi:", date_publication, "\n", 
        "Tag 1:", tag_1, "\n", 
        "Tag 2:", tag_2, "\n", 
        "Tag 3:", tag_3, "\n", 
        "Description:", description, "\n", 
        "Tags:", tags, "\n", 
        "Url Images:", images_url, "\n"
    )  
    
    return price, surface, nb_pieces, nb_chambres, location, prix_m2, date_publication, tag_1, tag_2, tag_3, description, tags, images_url
    

    

In [78]:
https://www.superimmo.com/annonces/achat-appartement-67m-bordeaux-33000-xw7bw5
https://www.superimmo.com/annonces/achat-appartement-paris-2eme-75002-xw8evz

SyntaxError: invalid decimal literal (1457235089.py, line 1)

In [80]:
price, surface, nb_pieces, nb_chambres, location, prix_m2, date_publication \
, tag_1, tag_2, tag_3, description, tags, images_url =  \
    get_details("https://www.superimmo.com/annonces/achat-appartement-78m-le-haillan-33185-xvzz53", driver)

Price: 295400 
 Surface: 5 
 Nb pieces: 4 
 Nb chambres: 3 
 Location: Le Haillan (33185) 
 Prix m2: 3763 
 Date Publi: ['10/06/2024'] 
 Tag 1: Cuisine de 5 m² 
 Tag 2:  
 Tag 3: 15 400 € TTC inclus dans le prix de vente. Barème d’honoraires : https://api.apimo.pro/uploads/agency/22481/fees_document.pdf?1687532692 
 Description: Maison de ville T4 à LE HAILLAN 33185
Dans une résidence sécurisée.
Maison de ville T4 de 79m² de 2011 se composant :
En RDC, une entrée sur séjour, une pièce de vie donnant sur le jardin de 43 m², une cuisine aménagée/équipée et un wc.
A l'étage, 3 chambres, une salle de bains et un wc.
La maison est en très bon état général.
Un garage est attenant + 2 places de parking privatives. 
 Tags: ['78,5 m² - 4 pièces - 3 chambres', 'Orientation : Nord', 'Année de construction : 2011', "Assainissement : tout à l'égout", 'Eau chaude individuelle : chaudière', 'Cuisine de 5 m²', 'Jardin (43 m²)', 'Fenêtres double vitrage', None, None, None, None, None, None, None, None,

In [ ]:
# Wait 10 secondes until we find a specific element (can be ID, XPATH, etc.)
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID = 'cnt')))

# Wait 10 secondes (more simple)
import time
time.sleep(10)

In [ ]:
#inputting text into a box
box = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input')
box.send_keys('web scraping')
box.send_keys(Keys.ENTER)


#clicking on a button
button = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[3]/center/input[1]')
button.click()
link = driver.find_element_by_xpath('//*[@id="rso"]/div[3]/div/div[1]/a/h3').click()
data_scraping = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/p[1]/a[1]').click()


#taking a screenshot
driver.save_screenshot('C:\Web Scraping course\screenshot.png')
driver.find_element_by_xpath('//*[@id="rso"]/div[3]/div/div[1]/a/h3').screenshot('C:\Web Scraping course\screenshot2.png')

#full example - uses inputting text into a box, clicking on a button, and taking a screenshot
driver = webdriver.Chrome('C:/Web Scraping course/chromedriver.exe')
driver.get('https://www.google.com/')
box = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input')
box.send_keys('giraffe')
box.send_keys(Keys.ENTER)
driver.find_element_by_xpath('//*[@id="hdtb-msb-vis"]/div[2]/a').click()
driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[3]/a[1]/div[1]/img').screenshot('C:\Web Scraping course\giraffe.png')

#self scrolling
driver.execute_script('return document.body.scrollHeight')
driver.execute_script('window.scrollTo(0,6000)')
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')


#wait times
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_condition as EC
import time

box = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input')
box.send_keys('giraffe')
box.send_keys(Keys.ENTER)
time.sleep(3)
driver.find_element_by_xpath('//*[@id="hdtb-msb-vis"]/div[2]/a').click()

element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'cntratet')))

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('df_iledefranceparis.csv')
df.head()

,Unnamed: 0,url,price,surface,nb_pieces,nb_chambres,lieu,prix_m2,date_publication,tag_1,tag_2,tag_3,description,tags,images_url
0,0,https://www.superimmo.com/annonces/achat-appar...,945 000,95,4,2.0,Paris 10ème (75010),9 947,10/02/2023,Étage 1/6 avec ascenseur,Chauffage individuel au fuel (convecteurs),Honoraires à la charge du vendeur.,"Double exposition pour cet appartement de 95,0...","['95 m² - 4 pièces - 2 chambres', 'Étage 1/6 a...",['https://photo.superimmo.com/photos/listings/...
1,1,https://www.superimmo.com/annonces/achat-appar...,375 000,41,2,1.0,Paris 13ème (75013),9 146,23/03/2023,Exclusivité,Dernier étage,Honoraires à la charge du vendeur.,Liberkeys vous propose cet appartement T2 de 4...,"['41 m² - 2 pièces - 1 chambre', 'Dernier étag...","[None, None, None, None, None, None, None, Non..."
2,2,https://www.superimmo.com/annonces/achat-appar...,625 000,54,3,2.0,Paris 9ème (75009),11 574,23/03/2023,NaN,NaN,Honoraires à la charge du vendeur.,"Au cœur du 9ème arrondissement de Paris, dans ...","['54 m² - 3 pièces - 2 chambres', 'Orientation...",['https://photo.superimmo.com/photos/listings/...
3,3,https://www.superimmo.com/annonces/achat-appar...,384 500,38,2,1.0,Paris 14ème (75014),10 118,10/10/2022,Étage 1/4,Cuisine aménagée,Honoraires à la charge du vendeur. Barèmes d’h...,"14ème arrondissement - Alesia/Didot, proche de...","['Disponible immédiatement', '38 m² - 2 pièces...",['https://photo.superimmo.com/photos/listings/...
4,4,https://www.superimmo.com/annonces/achat-appar...,885 000,74,3,2.0,Paris 17ème (75017),11 959,24/03/2023,Exclusivité,NaN,30 000 € TTC inclus dans le prix de vente - Pr...,entretenu par une gardienne à demeure.\nDécouv...,"['74 m² - 3 pièces - 2 chambres', 'Copropriété...",['https://photo.superimmo.com/photos/listings/...
